In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.feature_selection import mutual_info_regression
import numpy as np
import time

In [2]:
start_date = '2016-01-04'
end_date = '2020-03-01'

In [3]:
exchange = "btc-usd" 

In [4]:
data1 = pd.read_csv("generated_price_data/"+exchange+"_daily.csv")

In [5]:
um = "wiki_pageview" # usermetric: either wiki_pageview or trends 
ak = "Bitcoin" #either article name or a keyword name
tu = "daily" #time unit

In [6]:
data2 = pd.read_csv("generated_"+um+"_data/"+ak+"_"+tu+".csv")

In [7]:
## Filter data by dates
data1.index = [pd.Timestamp(t) for t in data1['day']]
data2.index = [pd.Timestamp(t) for t in data2['day']]


d1 = data1[start_date:end_date].copy()
d2 = data2[start_date:end_date].copy()

In [8]:
d1.drop_duplicates(subset=['day'],keep='first',inplace=True)
d2.drop_duplicates(subset=['day'],keep='first',inplace=True)

In [9]:
print(d1.shape)
print(d2.shape)

(1519, 8)
(1519, 17)


In [14]:
target = ['close']
feature_names = ['pageview','sma10','sma21'] #first element either trend or pageview

In [15]:
features_interested = ['log_','log_diff_','diff_ratio_']

In [16]:
data_dict = dict()

lag = 14 
window_size = [14,30,60]
i=0

In [17]:
st = time.time()
for fi in features_interested:
    t = [fi+t_ for t_ in target]
    fn = [fi+fn_ for fn_ in feature_names]
    for col1 in t:
        for col2 in fn:
            for w in window_size:
                for l in range(1,lag+1):
                    fs = np.zeros((1,w))
                    fs_ = d2.iloc[:-l][col2].values
                    
                    ts = np.zeros((1))
                    ts_ = d1.iloc[w+l:][col1].values
                    for s in range(d1.iloc[w+l:].shape[0]):
                        if np.isnan(ts_[s:s+1]).any() or np.isnan(fs_[s:s+w]).any():
                            continue
                        else:
                            ts = np.append(ts,ts_[s:s+1],axis=0) 
                            fs = np.append(fs,[fs_[s:s+w]],axis=0)
                    ts = ts[1:]
                    fs = fs[1:,:]
                    #feature series                    
                    #for f in range(fs_.shape[0]-w):
                    #    fs = np.append(fs,[fs_[f:f+w]],axis=0)
                    #fs = fs[1:,:]
                    try:
                        mi = mutual_info_regression(fs,ts).sum()
                    except:
                        mi = np.nan

                    data_dict[i] = {"window_size":w,
                                    "lag":l,
                                    "feature_1":col1,
                                    "feature_2":col2,
                                    "mutual_info":mi}

                    i += 1
ft = time.time()

In [18]:
ft-st

204.90998101234436

In [19]:
data = pd.DataFrame.from_dict(data_dict, "index")
data

,window_size,lag,feature_1,feature_2,mutual_info
0,14,1,log_close,log_pageview,8.030443
1,14,2,log_close,log_pageview,7.966397
2,14,3,log_close,log_pageview,7.953771
3,14,4,log_close,log_pageview,7.948264
4,14,5,log_close,log_pageview,7.948475
...,...,...,...,...,...
373,60,10,diff_ratio_close,diff_ratio_sma21,0.655282
374,60,11,diff_ratio_close,diff_ratio_sma21,0.649434
375,60,12,diff_ratio_close,diff_ratio_sma21,0.648863
376,60,13,diff_ratio_close,diff_ratio_sma21,0.629308


In [20]:
data['mutual_info'].dropna().shape

(378,)

In [21]:
file_name = 'generated_mutual_info_data/'+um+'/'
file_name += ak+'_'+exchange+'.csv'

In [22]:
file_name

'generated_mutual_info_data/wiki_pageview/Bitcoin_btc-usd.csv'

In [23]:
data.to_csv(file_name,index=False)